In [40]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import zscore
import matplotlib.pyplot as plt 

In [2]:
colnames = ['preg', 'glu', 'bp', 'sft', 'ins', 'bmi', 'dpf', 'age', 'outcome']
prima_df = pd.read_csv("prima-indians-diabetes.csv",names=colnames)

In [3]:
prima_df.head()

,preg,glu,bp,sft,ins,bmi,dpf,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X =  prima_df.drop("outcome", axis=1)
y =  prima_df.pop("outcome")

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
sc=StandardScaler()
X=sc.fit_transform(X)

C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [8]:
# instantiate learning model (k = 5)
knn = KNeighborsClassifier(n_neighbors=5)

# fitting the model
knn.fit(X_train, y_train)

# predict the response
pred = knn.predict(X_test)

In [9]:
# evaluate accuracy
print(metrics.accuracy_score(y_test, pred))

0.7705627705627706


Run grid search CV

In [10]:
from sklearn.model_selection import GridSearchCV
parameter={'n_neighbors':np.arange(1,100),'weights' : ['uniform','distance']}


In [11]:
GS=GridSearchCV(knn,parameter,cv=3,scoring='roc_auc')
GS.fit(X,y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  2, ..., 98, 99]), 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [19]:
GS.best_params_

{'n_neighbors': 97, 'weights': 'distance'}

In [20]:
# instantiate learning model (k = 5)
knn1 = KNeighborsClassifier(n_neighbors=97,weights='distance')

# fitting the model
knn1.fit(X_train, y_train)

# predict the response
pred1 = knn.predict(X_test)

In [21]:
# evaluate accuracy
print(metrics.accuracy_score(y_test, pred1))

0.7705627705627706


LR (Logistic regression)

In [22]:
from sklearn.linear_model import LogisticRegression # Logistics Regression

In [23]:
# Machine 
lg = LogisticRegression()
# Build the Model
lg.fit(X_train, y_train)

C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
pred_diabetes =lg.predict(X_test) 

In [25]:
from sklearn.metrics import precision_score, recall_score
recall_score(y_test, pred_diabetes)

0.527027027027027

In [26]:
precision_score(y_test, pred_diabetes)*100

70.9090909090909

Naive Bayes

In [27]:
from sklearn.naive_bayes import GaussianNB

In [28]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [29]:
# make predictions
expected = y_test
predicted = model.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83       157
           1       0.67      0.51      0.58        74

   micro avg       0.76      0.76      0.76       231
   macro avg       0.73      0.70      0.71       231
weighted avg       0.75      0.76      0.75       231

[[138  19]
 [ 36  38]]


In [30]:
Dt_model=DecisionTreeClassifier(criterion='entropy',random_state=0)

In [31]:
Dt_model2=DecisionTreeClassifier(max_depth=4,criterion='entropy',random_state=0)

In [32]:
Rf_model = RandomForestClassifier(n_estimators=57,criterion='entropy',random_state=0 )

In [33]:
bag_model=BaggingClassifier(n_estimators=10,random_state=0)##Fully grown decision tree

In [34]:
bag_model2=BaggingClassifier(n_estimators=10,random_state=0,base_estimator=Dt_model2)## Regularised decision tree

In [35]:
bag_LR=BaggingClassifier(n_estimators=5,random_state=0,base_estimator=lg)## LR

In [36]:
bag_KNN=BaggingClassifier(n_estimators=5,random_state=0,base_estimator=knn1)## KNN regularised

In [37]:
bag_NB=BaggingClassifier(n_estimators=4,random_state=0,base_estimator=model)## Naive bayes

In [41]:
GB_boost=GradientBoostingClassifier(n_estimators=100)

In [42]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
kf=KFold(n_splits=5,shuffle=True,random_state=0)
 

In [43]:
for model, name in zip([Dt_model,Dt_model2,bag_model,bag_model2,Rf_model,lg,bag_LR,knn1,bag_KNN,model,bag_NB,GB_boost], 
      ['Desicion Tree','Desicion Tree Regularised','Bagging Desicion Tree','Bagging Desicion Tree Reg','Random forest','Logistic regrssion','Logistic regrssion Bagged','KNN','KNN Bagged','Naive Bayes','Bagged Naive Bayes','Gradient Boost']):
    roc_auc=[]
    for train,test in kf.split(X,y):
        X_train,X_test=X[train,:],X[test,:]
        y_train,y_test=y.iloc[train],y.iloc[test]
        model.fit(X_train,y_train)
        y_predict=model.predict(X_test)
        #cm=metrics.confusion_matrix(Ytest,Y_predict)
        fpr,tpr, _ = roc_curve(y_test,y_predict)
        roc_auc.append(auc(fpr, tpr))
    print("AUC scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(roc_auc),
                                    np.var(roc_auc,ddof=1), name ))  

AUC scores: 0.69 (+/- 0.00223) [Desicion Tree]
AUC scores: 0.72 (+/- 0.00219) [Desicion Tree Regularised]
AUC scores: 0.70 (+/- 0.00279) [Bagging Desicion Tree]
AUC scores: 0.71 (+/- 0.00084) [Bagging Desicion Tree Reg]
AUC scores: 0.71 (+/- 0.00226) [Random forest]
AUC scores: 0.73 (+/- 0.00054) [Logistic regrssion]
AUC scores: 0.72 (+/- 0.00017) [Logistic regrssion Bagged]
AUC scores: 0.66 (+/- 0.00102) [KNN]


C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

AUC scores: 0.64 (+/- 0.00032) [KNN Bagged]
AUC scores: 0.71 (+/- 0.00237) [Naive Bayes]
AUC scores: 0.71 (+/- 0.00196) [Bagged Naive Bayes]
AUC scores: 0.73 (+/- 0.00239) [Gradient Boost]


In [44]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

1-roc_auc = Bias error 


In [45]:
from sklearn.ensemble import AdaBoostClassifier

In [46]:
LR_boost=AdaBoostClassifier(base_estimator=lg,n_estimators=100)


In [47]:
NB_boost=AdaBoostClassifier(base_estimator=model,n_estimators=100)


In [48]:
RF_boost=AdaBoostClassifier(base_estimator=Rf_model,n_estimators=100)


In [49]:
DT_boost=AdaBoostClassifier(base_estimator=Dt_model,n_estimators=100)

In [50]:
DTR_boost=AdaBoostClassifier(base_estimator=Dt_model2,n_estimators=100)

In [52]:
from sklearn.ensemble import VotingClassifier

In [53]:
#Stacked Model
stacked = VotingClassifier(estimators = [('Bagged_LR',bag_LR),('BoostedRF',RF_boost ), ('GBoost',GB_boost )],voting='hard')

In [56]:
#Stacked Model
stacked1 = VotingClassifier(estimators = [('Bagged_LR',bag_LR),('BoostedRF',RF_boost ), ('GBoost',GB_boost )],voting='soft')

In [57]:
for model, name in zip([LR_boost,NB_boost,RF_boost,DT_boost,DTR_boost,stacked,stacked1], 
      ['Boosted_LG','Boosted_NB','Boosted_RandomForest','Boosted_DT','Boosted_DTRegularised','Stacked Model Hard','Stacked Model Soft']):
    roc_auc=[]
    for train,test in kf.split(X,y):
        X_train,X_test=X[train,:],X[test,:]
        y_train,y_test=y.iloc[train],y.iloc[test]
        model.fit(X_train,y_train)
        y_predict=model.predict(X_test)
        #cm=metrics.confusion_matrix(Ytest,Y_predict)
        fpr,tpr, _ = roc_curve(y_test,y_predict)
        roc_auc.append(auc(fpr, tpr))
    print("AUC scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(roc_auc),
                                    np.var(roc_auc,ddof=1), name ))  

AUC scores: 0.72 (+/- 0.00059) [Boosted_LG]
AUC scores: 0.69 (+/- 0.00222) [Boosted_NB]
AUC scores: 0.72 (+/- 0.00240) [Boosted_RandomForest]
AUC scores: 0.69 (+/- 0.00169) [Boosted_DT]
AUC scores: 0.70 (+/- 0.00267) [Boosted_DTRegularised]
AUC scores: 0.74 (+/- 0.00214) [Stacked Model Hard]
AUC scores: 0.74 (+/- 0.00146) [Stacked Model Soft]
